## 134653 ICS 4A
The following code is an example of the code optimization phase in the compiler process. This has been achieved here by constant folding and dead code elimination.

In [15]:
!pip install astor

In [2]:
import ast #The Python Abstract Syntax Tree (AST) module
import copy
import astor

In [11]:
#Constant Folding Implementation
class ConstantFolding(ast.NodeTransformer):
    def __init__(self):
        self.constants = {}

    def visit_Assign(self, node):
        # Record assignments to variables with constant values
        if len(node.targets) == 1 and isinstance(node.targets[0], ast.Name):
            target_var = node.targets[0].id
            if isinstance(node.value, (ast.Constant, ast.NameConstant)):
                try:
                    value = eval(astor.to_source(node.value))
                    self.constants[target_var] = value
                except:
                    pass  # Ignore errors during evaluation

        return self.generic_visit(node)

    def visit_BinOp(self, node):
        # Check if both operands are constants or variables with constant values
        if (
            isinstance(node.left, (ast.Constant, ast.NameConstant)) and
            isinstance(node.right, (ast.Constant, ast.NameConstant))
        ):
            # Evaluate the expression at compile-time
            try:
                result = eval(astor.to_source(node), self.constants)
                return ast.Constant(value=result, kind=None)
            except:
                pass  # Ignore errors during evaluation

        return self.generic_visit(node)


In [8]:
#Dead Code Elimination Implementation
class DeadCodeElimination(ast.NodeTransformer):
    def __init__(self):
        self.live_variables = set()

    def visit_Assign(self, node):
        # Record live variables from assignments
        for target in node.targets:
            if isinstance(target, ast.Name):
                self.live_variables.add(target.id)
        return self.generic_visit(node)

    def visit_Name(self, node):
        # Remove references to dead variables
        if isinstance(node, ast.Name) and node.id not in self.live_variables:
            return ast.Pass()
        return self.generic_visit(node)

In [12]:
#Combined Optimizer
class CombinedOptimizer(ConstantFolding, DeadCodeElimination):
    def optimize_code(self, source_code):
        tree = ast.parse(source_code)

        # Perform constant folding
        constant_folded_tree = copy.deepcopy(tree)
        constant_folding_optimizer = ConstantFolding()
        constant_folded_tree = constant_folding_optimizer.visit(constant_folded_tree)

        # Perform dead code elimination
        dead_code_elimination_optimizer = DeadCodeElimination()
        optimized_tree = dead_code_elimination_optimizer.visit(constant_folded_tree)

        return astor.to_source(optimized_tree)

In [14]:
#  Example in use
source_code = """
x = 2 + 3
y = x * 4
z = 5
# Dead code example

"""

optimizer = CombinedOptimizer()
optimized_code = optimizer.optimize_code(source_code)

# Section 5: Print Original and Optimized Code
print("Original Code:")
print(source_code)
print("\nOptimized Code:")
print(optimized_code)

Original Code:

x = 2 + 3
y = x * 4
z = 5
# Dead code example



Optimized Code:
x = 5
y = x * 4
z = 5

